In [ ]:
import pandas as pd
from icalendar import Calendar, Event
from calendar_utils import parse_date, parse_time,find_sub_group
from ical_element import Subject

In [ ]:
excel_schedule=pd.read_excel('autumn.xlsx', index_col=0)
CURRENT_YEAR = 2022

In [ ]:
df = excel_schedule.copy()
df = df.reset_index(drop=True).drop([0,1,2], axis=0)
df.columns = ['Time', 'Date', 'SubjectName', 'Place', 'Teacher']

In [ ]:
# Парсим даты
df['Dates'] = df['Date'].apply(lambda x: parse_date(x, CURRENT_YEAR))

In [ ]:
# Распаковываем даты, соединяем с исходным DF
df_new = df.groupby(['Date', 'SubjectName', 'Time'])['Dates'].apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_3', axis = 1)
df_new.columns = ['Date', 'SubjectName', 'Time', 'DateStart','DateEnd', 'RepeatTime', 'RepeatType']
df = pd.merge(df, df_new, on=['Date', 'SubjectName', 'Time'])

In [ ]:
# Убираем все траектории (Ангийский и тд)
# df = df[~df['SubjectName'].str.startswith('Траектория')]
# df = df[df['SubjectName'].str.match('((^(?!Траектория).*)(^(?!Русский язык).*))|(Траектория 3 .*)')]

In [ ]:
# Находим подгруппы
df['SubGroup'] = df['SubjectName'].apply(lambda x: find_sub_group(x))

In [ ]:
# Парсим время и удаляем не нужные столбцы
df[['TimeStart', 'TimeEnd']] = pd.DataFrame(df['Time'].apply(parse_time).tolist(), index=df.index)
df = df.drop(['Dates', 'Time'], axis=1)

In [ ]:
# Сохранение в csv
# df.to_csv('autumn.csv')

In [ ]:
# Создаем календарь
cal = Calendar()
cal.add('prodid', '-//Karpovich//Karpovich Alex//EN')

In [ ]:
# Добавляем туда события
for idx, item in df.iterrows():
    s = Subject.from_series(item)
    e = s.to_event()
    cal.add_component(e)

In [ ]:
# Сохраняем календарь. Ура
with open('autumn_2022.ics', 'wb') as f:
    data = cal.to_ical()
    f.write(data)